In [17]:
# https://rosettacode.org/wiki/Sorting_algorithms/Insertion_sort#Python

import numpy as np
from line_profiler import LineProfiler
from pysr import PySRRegressor
import random
import os
import sys
import math
import pickle

## Insertionsort Algorithm

In [18]:
def insertion_sort(L): # x^2
    for i in range(1, len(L)):
        j = i-1 
        key = L[i]
        while j >= 0 and L[j] > key:
           L[j+1] = L[j]
           j -= 1
        L[j+1] = key

## Frequency Count Method

In [19]:
X_y = []
i=50
for n in range(2,i+1): # started with 2 itens until 50 itens in the list
  lprofiler = LineProfiler()
  lp_wrapper = lprofiler(insertion_sort)

  # create a list with random number between 1 to 10000
  # with n number in the list
  input = random.choices(range(10000), k=n)
  lp_wrapper(input)

  stats = lprofiler.get_stats()
  line_numbers = []
  hits = []

  for line in stats.timings.values():
    for i in line:
      line_numbers.append(i[0])
      hits.append(i[1])

  X_y.append([n, sum(hits)])

dados = np.array(X_y)

X = dados[:, 0]
y = dados[:, 1]
X_reshaped = X.reshape(-1, 1)

In [20]:
resultados_com_menor_loss = []
repeat = 10
registros = []
unary_operators_list = ["log", "square", "cube", "sqrt", "round", "exp", "abs"]

original_stdout = sys.stdout

with open(os.devnull, 'w') as devnull:
  sys.stdout = devnull

  for i in range(repeat):

    # first combination
    reg1 = PySRRegressor(
      #binary_operators=["*", "+"],
      unary_operators=unary_operators_list
    )

    fit1 = reg1.fit(X_reshaped, y)
    best_program1 = fit1.get_best()

    registro1 = []
    for index, value in enumerate(best_program1):
      registro1.append(value)
      
    registros.append(registro1)

    # second combination
    #reg2 = PySRRegressor(
    #  binary_operators=["*"],
    #  unary_operators=unary_operators_list
    #)
#
    #fit2 = reg2.fit(X_reshaped, y)
    #best_program2 = fit2.get_best()
#
    #registro2 = []
    #for index, value in enumerate(best_program2):
    #  registro2.append(value)
    #registros.append(registro2)
#
    ## third combinarion
    #reg3 = PySRRegressor(
    #  binary_operators=["+"],
    #  unary_operators=unary_operators_list
    #)
#
    #fit3 = reg3.fit(X_reshaped, y)
    #best_program3 = fit3.get_best()
#
    #registro3 = []
    #for index, value in enumerate(best_program3):
    #  registro3.append(value)
    #registros.append(registro3)
    
sys.stdout = original_stdout

[ Info: Started!
0.0%┣                                              ┫ 0/600 [00:00<00:-4, -0s/it]Expressions evaluated per second: [.....]. Head worker occupation: 0.0%         Press 'q' and then <enter> to stop execution early.                             Hall of Fame:                                                                   ---------------------------------------------------------------------------------------------------                                                             Complexity  Loss       Score     Equation                                       1           1.002e+06  1.594e+01  y = x₀                                        2           2.536e+04  3.677e+00  y = square(x₀)                                4           1.878e+04  1.503e-01  y = (square(x₀) - x₀)                         6           1.395e+04  1.487e-01  y = ((square(x₀) - x₀) - x₀)                  8           9.453e+03  1.945e-01  y = ((square(x₀ - 1.0874) - x₀) - x₀)         10          8.762e+03  

In [21]:
registros_ = registros
file_pickle = 'dados_v3.pck'

In [22]:
for i in registros_:
  loss = i[1]
  score = i[2]
  complexity = i[0]
  w = (loss * score)/complexity
  if math.isnan(w):
    i.append(0)
  else:
    i.append(w)

lista_melhor_valor = max(registros_, key=lambda x: x[6])

## Save result

In [23]:
def salvar_dados(dados, key, arquivo):
  if os.path.exists(arquivo):
    with open(arquivo, 'rb') as f:
      dados_exist = pickle.load(f)
  else:
    dados_exist = {}
    
  valor_original = dados_exist.get(key)
  if valor_original == None:
    dados_exist.update({key: [dados]})
  else:
    if isinstance(valor_original, list):
      valor_original.append(dados)
    else:
      dados_exist.update({key: [dados]})

  with open(arquivo, 'wb') as f:
    pickle.dump(dados_exist, f)

caminho_arquivo = file_pickle
novos_dados = lista_melhor_valor[0:3] + [lista_melhor_valor[4]]

salvar_dados(novos_dados, 'insertionsort_N^2', caminho_arquivo)

In [24]:
def carregar_dados(arquivo):
    # Carrega os dados do arquivo pickle
    with open(arquivo, 'rb') as f:
        dados = pickle.load(f)
    return dados

caminho_arquivo = file_pickle
dados_carregados = carregar_dados(caminho_arquivo)

print("Conteúdo do arquivo pickle:")
for k, v in dados_carregados.items():
  if k == 'insertionsort_N^2':
    print('\u25CF', k)
    for index, item in enumerate(v):
      if index == len(v)-1:
        print('└─', item)
      else:
        print('├─', item)
    print('==========================')

Conteúdo do arquivo pickle:
● insertionsort_N^2
├─ [4, 8265.896, 0.8543297027289066, 0.83297912379361*x0**2]
├─ [6, 4116.6875, 0.4126062097274302, 10.314309043464*(0.258150714017089*x0 + 1)**2]
└─ [4, 7971.757, 0.5787193892087205, 0.886050969757211*x0**2]
